## 먼저 실행하기!

In [1]:
import sqlite3
from prettytable import from_db_cursor

# SQLite 데이터베이스에 연결하기 위해 connect 생성
con = sqlite3.connect('./database/example.db')
con.row_factory = sqlite3.Row

# cursor 생성
cur = con.cursor()

# 테이블이 이미 있다면 삭제
cur.execute("DROP TABLE IF EXISTS student")

# 데이터 삽입할 테이블 생성
cur.execute("""
    CREATE TABLE student(
        student_id INTEGER PRIMARY KEY,
        first_name text DEFAULT 'Anonymous',
        last_name text DEFAULT 'Anonymous',
        age integer DEFAULT 0
    )
    """)

## INSERT
테이블에 레코드를 추가합니다

`INSERT 테이블(컬럼) VALEUS (데이터)`

컬럼의 수와 데이터의 수가 일치해야 합니다.

In [3]:
cur.execute("""
            INSERT INTO student (first_name, last_name, age)
            VALUES ('Harry', 'Potter', 13)
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)

student_id,first_name,last_name,age
1,Harry,Potter,13


In [4]:
# 컬럼명을 적은 순서대로 데이터가 들어감
cur.execute("""
            INSERT INTO student (age, last_name, first_name)
            VALUES ('Train', 'Tomas', 80)
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)

student_id,first_name,last_name,age
1,Harry,Potter,13
2,80,Tomas,Train


In [5]:
# 컬럼 수와 데이터 수가 일치하지 않으면 에러 발생
cur.execute("""
            INSERT INTO student (first_name, last_name, age)
            VALUES ('Sponge', 'Bob')
            """)

OperationalError: 2 values for 3 columns

In [6]:
# NULL이 들어가더라도 데이터 수를 맞춰줘야 함
cur.execute("""
            INSERT INTO student (first_name, last_name, age)
            VALUES ('Sponge', 'Bob', NULL)
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)


student_id,first_name,last_name,age
1,Harry,Potter,13
2,80,Tomas,Train
3,Sponge,Bob,None


In [7]:
# 컬럼을 생략하면 스키마 순서대로 데이터 입력됨
cur.execute("""
            INSERT INTO student
            VALUES (4, 'Sponge', 'Bob', 52)
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)

student_id,first_name,last_name,age
1,Harry,Potter,13
2,80,Tomas,Train
3,Sponge,Bob,None
4,Sponge,Bob,52


In [8]:
# 테이블 생성 시 설정했던 Default 값으로 데이터를 저장할 수도 있음
cur.execute("""
            INSERT INTO student
            DEFAULT VALUES
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)

student_id,first_name,last_name,age
1,Harry,Potter,13
2,80,Tomas,Train
3,Sponge,Bob,None
4,Sponge,Bob,52
5,Anonymous,Anonymous,0


## UPDATE
데이터를 수정하고 싶을 때 사용합니다  
`UPDATE 테이블 SET 데이터 WHERE 조건`

In [9]:
cur.execute("""
            UPDATE student
            SET age = 30
            WHERE student_id = 3
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)

student_id,first_name,last_name,age
1,Harry,Potter,13
2,80,Tomas,Train
3,Sponge,Bob,30
4,Sponge,Bob,52
5,Anonymous,Anonymous,0


In [10]:
# 조건 설정할 때 주의해야함. 원하지 않는 레코드가 함께 수정될수도 있음
cur.execute("""
            UPDATE student
            SET age = 15
            WHERE first_name = 'Sponge'
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)

student_id,first_name,last_name,age
1,Harry,Potter,13
2,80,Tomas,Train
3,Sponge,Bob,15
4,Sponge,Bob,15
5,Anonymous,Anonymous,0


In [11]:
# 가능하면 유니크한 값으로 조작하는 것이 좋음
cur.execute("""
            UPDATE student
            SET age = 10, last_name = 'Song'
            WHERE student_id = 4
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)

student_id,first_name,last_name,age
1,Harry,Potter,13
2,80,Tomas,Train
3,Sponge,Bob,15
4,Sponge,Song,10
5,Anonymous,Anonymous,0


## DELETE
데이터를 삭제하고싶을 때 사용합니다.
`DELETE FROM 테이블 WHERE 조건`

In [12]:
cur.execute("""
            DELETE
            FROM student
            WHERE student_id IN (2, 5)
            """)

cur.execute("""
            SELECT * 
            FROM student
            """)

from_db_cursor(cur)

student_id,first_name,last_name,age
1,Harry,Potter,13
3,Sponge,Bob,15
4,Sponge,Song,10
